In [14]:
# Choose pairs based on correlation

# Imports, read and preview data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import linregress

df = pd.read_csv("../../data/etf_prices.csv")
df.set_index("Timestamp", inplace=True)

nan_percentages = df.isna().mean()[1:] * 100

valid_etfs = nan_percentages[nan_percentages == 0]
df = df[valid_etfs.index]

valid_columns = df.columns[df.iloc[0].notna() & df.iloc[-1].notna()]
df = df[valid_columns]

df_returns = df.pct_change().iloc[1:]

In [15]:
correlation_matrix = df.corr()

# Unstack the correlation matrix to a Series
correlation_pairs = correlation_matrix.unstack()

# Drop self-correlations (correlation of an asset with itself)
correlation_pairs = correlation_pairs[correlation_pairs != 1]

# Convert the multi-index Series to DataFrame to better handle potential issues
correlation_pairs_df = correlation_pairs.reset_index()
correlation_pairs_df.columns = ['Asset1', 'Asset2', 'Correlation']

# Drop duplicate pairs (keep only one of each pair)
# Sort by 'Asset1' and 'Asset2' to ensure pairs are consistently ordered
correlation_pairs_df = correlation_pairs_df[correlation_pairs_df['Asset1'] < correlation_pairs_df['Asset2']]

# Sort the values to find the highest correlations
sorted_correlation_pairs = correlation_pairs_df.sort_values(by='Correlation', ascending=False)

# Select the top 19 pairs with the highest correlation
top_19_correlation_pairs = sorted_correlation_pairs.head(19)

# Convert the top 19 pairs to a list of lists
top_19_pairs_list = top_19_correlation_pairs[['Asset1', 'Asset2']].values.tolist()

# Display the top 19 pairs as a list of lists
print(top_19_pairs_list)



[['USIG', 'VTC'], ['EMB', 'VWOB'], ['IGSB', 'VCSH'], ['VONE', 'VTHR'], ['SMH', 'SOXX'], ['IXUS', 'VXUS'], ['TLT', 'VGLT'], ['IGIB', 'VCIT'], ['ESGU', 'VONE'], ['CATH', 'VONE'], ['ACWX', 'IXUS'], ['MBB', 'VMBS'], ['IGSB', 'SUSB'], ['SUSC', 'USIG'], ['ESGU', 'VTHR'], ['CFA', 'FEX'], ['ACWX', 'VXUS'], ['CATH', 'ESGU'], ['SUSC', 'VTC']]


In [16]:
# Compute spread
spread_stats = {}


for pair in top_19_pairs_list:
    ticker_1 = pair[0]
    ticker_2 = pair[1]

    ticker_series_1 = df[ticker_1]
    ticker_series_2 = df[ticker_2]

    slope, intercept, _, _, _ = linregress(ticker_series_1, ticker_series_2)
    spread = ticker_series_2 - (slope * ticker_series_1 + intercept)

    spread_mean = np.mean(spread.values)
    spread_std = np.std(spread.values)

    spread_stats[f"{ticker_1}_{ticker_2}"] = {
        "slope": slope,
        "intercept": intercept,
        "mean": spread_mean,
        "std": spread_std
    }


{'USIG_VTC': {'slope': 1.5492893773103313, 'intercept': -1.8535042751858697, 'mean': 3.3901712229076677e-15, 'std': 0.15275155249839034}, 'EMB_VWOB': {'slope': 0.6787847122137008, 'intercept': 3.6024836544172842, 'mean': -6.8732238491826685e-15, 'std': 0.1872106311094548}, 'IGSB_VCSH': {'slope': 1.5018278431871739, 'intercept': 0.4093135015042151, 'mean': 2.1084078564384673e-14, 'std': 0.06931909761440742}, 'VONE_VTHR': {'slope': 0.9875798030525276, 'intercept': 0.253816751293499, 'mean': -2.380859972982645e-14, 'std': 0.7783823359904095}, 'SMH_SOXX': {'slope': 1.1402095039979951, 'intercept': 2.2081009528764923, 'mean': -1.4025091908467338e-14, 'std': 1.0620759006222757}, 'IXUS_VXUS': {'slope': 0.8815331877363543, 'intercept': 0.9833013743124042, 'mean': 2.701300814599945e-15, 'std': 0.2230250488944103}, 'TLT_VGLT': {'slope': 0.60021770183031, 'intercept': 1.415290274469669, 'mean': -7.430512269386669e-16, 'std': 0.4105055683989476}, 'IGIB_VCIT': {'slope': 1.602668638914883, 'intercep

In [ ]:
df = pd.read_csv("../../data/etf_prices_backtest.csv")
df.set_index("Timestamp", inplace=True)

nan_percentages = df.isna().mean()[1:] * 100

valid_etfs = nan_percentages[nan_percentages == 0]
df = df[valid_etfs.index]

valid_columns = df.columns[df.iloc[0].notna() & df.iloc[-1].notna()]
df = df[valid_columns]

df_returns = df.pct_change().iloc[1:]

for pair in top_19_pairs_list:
    ticker_1 = pair[0]
    ticker_2 = pair[1]

    ticker_series_1 = df[ticker_1]
    ticker_series_2 = df[ticker_2]

    spread = ticker_series_2 - (spread_stats[f"{ticker_1}_{ticker_2}"]["slope"] * ticker_series_1 + spread_stats[f"{ticker_1}_{ticker_2}"]["intercept"])

    upper_bound = spread_stats[f"{ticker_1}_{ticker_2}"]["mean"] + spread_stats[f"{ticker_1}_{ticker_2}"]["std"]
    lower_bound = spread_stats[f"{ticker_1}_{ticker_2}"]["mean"] - spread_stats[f"{ticker_1}_{ticker_2}"]["std"]

    # Generate signals & compute P&L

In [ ]:
# Generate P&L and success metrics